In [1]:
import tensorflow as tf
import numpy as np
import collections
import random
import itertools

In [2]:
# Target log path
logs_path = '/home/shawn/chat_bot_tf'
writer = tf.summary.FileWriter(logs_path)



# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000


# on agserver embedded
sess_host = None
#sess_host = 'grpc://genruckus:2222'


In [3]:
def readfile(name):
    with open(name) as file:
        lines = file.readlines()
    #lines = [line.strip() for line in lines]
    paras = [line.strip() for line in lines]
    #print (paras)
    
    words = []
    words.extend([para.split() for para in paras])

    words = list(itertools.chain(*words))
    #words = paras.split()
    words = [word.lower() for word in words]
    #print(words)
    

    return words

In [4]:
# USEFUL SHELL COMMAND:
# sed -e 's/\W/ /g' -e 's/ \+/ /g' -e 's/\([^ ]\)$/\1 /' pg1727.txt | tr -d '\n'

words = []
words.extend(readfile('/pub/writings/jefferson.txt'))
words.extend(readfile('/pub/writings/lincoln.txt'))
#words.extend(readfile('/pub/writings/hamlet.txt'))
words.extend(readfile('/pub/writings/cantebury_tales.txt'))
words.extend(readfile('/pub/writings/odyssy.txt'))

#print(words)
print(len(words))

960868


In [5]:
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary








In [6]:

word_dict, rev_word_dict = build_dataset(words)
print (word_dict)
print (rev_word_dict)

print(len(word_dict))


{'the': 0, 'of': 1, 'and': 2, 'to': 3, 'that': 4, 'in': 5, 'a': 6, 'i': 7, 'it': 8, 'he': 9, 'is': 10, 'for': 11, 'as': 12, 'be': 13, 'his': 14, 'you': 15, 'not': 16, 'this': 17, 'was': 18, 'have': 19, 'with': 20, 'by': 21, 'all': 22, 'but': 23, 'or': 24, 'on': 25, 'which': 26, 'they': 27, 'so': 28, 'will': 29, 'him': 30, 'at': 31, 'my': 32, 'me': 33, 'from': 34, 'had': 35, 'we': 36, 'if': 37, 'no': 38, 's': 39, 'them': 40, 'your': 41, 'her': 42, 'are': 43, 'their': 44, 'there': 45, 'one': 46, 'may': 47, 'do': 48, 'would': 49, 'now': 50, 'when': 51, 'any': 52, 'general': 53, 'shall': 54, 'were': 55, 'she': 56, 'upon': 57, 'an': 58, 'has': 59, 'who': 60, 'been': 61, 'our': 62, 'what': 63, 'states': 64, 'can': 65, 'said': 66, 'such': 67, 'man': 68, 'then': 69, 'other': 70, 'lincoln': 71, 'more': 72, 'men': 73, 'than': 74, 'us': 75, 'say': 76, 'some': 77, 'made': 78, 'well': 79, 'should': 80, 'out': 81, 'people': 82, 'time': 83, 'great': 84, 'these': 85, 'about': 86, 'day': 87, 'right': 8

In [7]:

vocab_size = len(word_dict)
n_input = 3
# number of units in RNN cell
n_hidden = 512


x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])



# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

In [8]:
def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 1-layer LSTM with n_hidden units.
    rnn_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = tf.contrib.rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']


In [ ]:
pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Prepare saver
#saver = tf.train.Saver()

#"""create summary op"""
#merged = tf.summary.merge_all

#log.debug("starting train writer: {}".format(save_path))
#train_writer = tf.summary.FileWriter(save_path, sess.graph)
#log.info("train writer started")


In [ ]:
# Launch the graph
with tf.Session(sess_host) as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(words)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [word_dict[ str(words[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[word_dict[str(words[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [words[i] for i in range(offset, offset + n_input)]
            symbols_out = words[offset + n_input]
            symbols_out_pred = rev_word_dict[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
#    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [word_dict[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,rev_word_dict[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")
            

Iter= 1000, Average Loss= 13.254476, Average Accuracy= 4.30%
['its', 'outspoken', 'tone'] - [on] vs [and]
Iter= 2000, Average Loss= 9.715195, Average Accuracy= 5.80%
['to', 'present', 'these'] - [papers] vs [not]
Iter= 3000, Average Loss= 9.278288, Average Accuracy= 5.20%
['when', 'he', 'an'] - [old] vs [of]
Iter= 4000, Average Loss= 9.192131, Average Accuracy= 5.80%
['his', 'position', 'at'] - [a] vs [of]
Iter= 5000, Average Loss= 9.353323, Average Accuracy= 5.70%
['thenceforth', 'submitted', 'to'] - [lincoln] vs [he]
Iter= 6000, Average Loss= 9.061238, Average Accuracy= 6.50%
['secretaries', 'a', 'funny'] - [chapter] vs [orpheus]
